In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [97]:
df = pd.read_csv('../datasets/raw_data/spotify-2023.csv', encoding='ISO-8859-1')
print(df.shape)
df.head()

(953, 24)


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


# Task: top 100 streamed song classification

In [77]:
df['streams'] = df['streams'] = pd.to_numeric(df['streams'], errors='coerce')
df.describe()
df = df.sort_values(by=['in_spotify_charts', 'in_spotify_playlists'], ascending=[False, False])
df = df.reset_index(drop=True)
df['top_countdown'] = range(1, len(df) + 1)

df['top_100'] = df['top_countdown'].apply(lambda x: 1 if x <= 300 else 0)

In [78]:
df[['in_spotify_charts', 'streams', 'in_spotify_playlists', 'top_countdown', 'top_100']]
df.describe()

,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_charts,bpm,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,top_countdown,top_100
count,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,9.520000e+02,953.000000,953.000000,953.000000,953.000000,953.00000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000
mean,1.556139,2018.238195,6.033578,13.930745,5200.124869,12.009444,5.141374e+08,67.812172,51.908709,2.666317,122.540399,66.96957,51.431270,64.279119,27.057712,1.581322,18.213012,10.131165,477.000000,0.314795
std,0.893044,11.116218,3.566435,9.201949,7897.608990,19.575992,5.668569e+08,86.441493,50.630241,6.035599,28.057802,14.63061,23.480632,16.550526,25.996077,8.409800,13.711223,9.912888,275.251703,0.464678
min,1.000000,1930.000000,1.000000,1.000000,31.000000,0.000000,2.762000e+03,0.000000,0.000000,0.000000,65.000000,23.00000,4.000000,9.000000,0.000000,0.000000,3.000000,2.000000,1.000000,0.000000
25%,1.000000,2020.000000,3.000000,6.000000,875.000000,0.000000,1.416362e+08,13.000000,7.000000,0.000000,100.000000,57.00000,32.000000,53.000000,6.000000,0.000000,10.000000,4.000000,239.000000,0.000000
50%,1.000000,2022.000000,6.000000,13.000000,2224.000000,3.000000,2.905309e+08,34.000000,38.000000,0.000000,121.000000,69.00000,51.000000,66.000000,18.000000,0.000000,12.000000,6.000000,477.000000,0.000000
75%,2.000000,2022.000000,9.000000,22.000000,5542.000000,16.000000,6.738690e+08,88.000000,87.000000,2.000000,140.000000,78.00000,70.000000,77.000000,43.000000,0.000000,24.000000,11.000000,715.000000,1.000000
max,8.000000,2023.000000,12.000000,31.000000,52898.000000,147.000000,3.703895e+09,672.000000,275.000000,58.000000,206.000000,96.00000,97.000000,97.000000,97.000000,91.000000,97.000000,64.000000,953.000000,1.000000


In [79]:
df.isna().sum()
df['key'] = df['key'].fillna("NA")

In [98]:
key_dummies = pd.get_dummies(df['key'], prefix='key_YN').astype(int) 
df['mode'] = df['mode'].map({'Major': 1, 'Minor': 0})

In [99]:
base_df = pd.concat([df, key_dummies], axis=1)

In [100]:
base_df = base_df[['artist_count', 'bpm', 'danceability_%', 'valence_%', 'energy_%',
    'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%', 'key_YN_A', 
    'key_YN_A#', 'key_YN_B', 'key_YN_C#', 'key_YN_D', 'key_YN_D#', 'key_YN_E',
    'key_YN_F', 'key_YN_F#', 'key_YN_G', 'key_YN_G#', 'mode']]
base_df

,artist_count,bpm,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,key_YN_A,...,key_YN_B,key_YN_C#,key_YN_D,key_YN_D#,key_YN_E,key_YN_F,key_YN_F#,key_YN_G,key_YN_G#,mode
0,2,125,80,89,83,31,0,8,4,0,...,1,0,0,0,0,0,0,0,0,1
1,1,92,71,61,74,7,0,10,4,0,...,0,1,0,0,0,0,0,0,0,1
2,1,138,51,32,53,17,0,31,6,0,...,0,0,0,0,0,1,0,0,0,1
3,1,170,55,58,72,11,0,11,15,1,...,0,0,0,0,0,0,0,0,0,1
4,1,144,65,23,80,14,63,11,6,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,1,144,60,24,39,57,0,8,3,1,...,0,0,0,0,0,0,0,0,0,1
949,1,166,42,7,24,83,1,12,6,0,...,0,0,0,0,0,0,1,0,0,1
950,2,92,80,81,67,4,0,8,6,0,...,0,1,0,0,0,0,0,0,0,1
951,3,97,82,67,77,8,0,12,5,0,...,0,1,0,0,0,0,0,0,0,1


In [101]:
from sklearn.model_selection import train_test_split

X = base_df.drop(columns=['mode'])
y = base_df['mode']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0, k_neighbors=5)
X_train,y_train =os.fit_resample(X_train, y_train)

c:\Users\JohnnyKeepWalking\.conda\envs\hw1_conda\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [102]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# conf_matrix = confusion_matrix(y_test, y_pred)
# print("Confusion Matrix:")
# print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

roc_auc = roc_auc_score(y_test, y_pred)
print(f"\nROC-AUC Score: {roc_auc:.2f}")

Accuracy: 0.50

Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.34      0.37        83
           1       0.55      0.63      0.59       108

    accuracy                           0.50       191
   macro avg       0.48      0.48      0.48       191
weighted avg       0.49      0.50      0.49       191


ROC-AUC Score: 0.48


In [103]:
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', C=1, gamma=0.61, random_state=42)
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.5654450261780105
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        83
           1       0.57      1.00      0.72       108

    accuracy                           0.57       191
   macro avg       0.28      0.50      0.36       191
weighted avg       0.32      0.57      0.41       191



c:\Users\JohnnyKeepWalking\.conda\envs\hw1_conda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\JohnnyKeepWalking\.conda\envs\hw1_conda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\JohnnyKeepWalking\.conda\envs\hw1_conda\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [107]:
from sklearn.ensemble import GradientBoostingClassifier

gbdt = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, subsample=1)
gbdt.fit(X_train, y_train)

y_pred = gbdt.predict(X_train)

print("Accuracy:", accuracy_score(y_pred, y_train))
print("Classification Report:\n", classification_report(y_pred, y_train))

Accuracy: 0.8556430446194225
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.91      0.81       258
           1       0.95      0.83      0.88       504

    accuracy                           0.86       762
   macro avg       0.84      0.87      0.85       762
weighted avg       0.87      0.86      0.86       762

